In [1]:
import json
import os

import numpy as np
import shutil
from tqdm import tqdm

In [2]:
with open("/home/vblot/multiaccurate_cp/annotations/instances_val2014.json") as f:
    data = json.load(f)

In [3]:
PATH = "/home/vblot/multiaccurate_cp/val2014"
OUTPUT_PATH = "/home/vblot/multiaccurate_cp/data/coco/02_prepared_data"


In [4]:
def coco91_to_coco80_class():
    """
    Converts 91-index COCO class IDs to 80-index COCO class IDs.

    Returns:
        (list): A list of 91 class IDs where the index represents the 80-index class ID and the value is the
            corresponding 91-index class ID.
    """
    return [
        0,1,2,3,4,5,6,7,8,9,10,None,11,12,13,14,15,16,17,18,19,20,21,22,23,
        None,24,25,None,None,26,27,28,29,30,31,32,33,34,35,36,37,38,39,None,
        40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,None,60,
        None,None,61,None,62,63,64,65,66,67,68,69,70,71,72,None,73,74,75,76,
        77,78,79,None,
    
    ]

In [5]:
len(coco91_to_coco80_class())

91

In [6]:
def find_img_name_from_id(img_id):
    for img in data["images"]:
        if img["id"] == img_id:
            return img["file_name"]

In [7]:
annotations = {}
coco_convertion = coco91_to_coco80_class()
for annotation in tqdm(data["annotations"]):
    annot_img_id = annotation["image_id"]
    img_name = find_img_name_from_id(annot_img_id)
    if img_name not in annotations:
        annotations[img_name] = np.zeros(80)
    annotation90 = annotation["category_id"]
    annotation80 = coco_convertion[annotation90 - 1]
    annotations[img_name][annotation80] = 1

100%|██████████| 291875/291875 [02:27<00:00, 1981.19it/s]


In [8]:
all_annots = []

for img_name, annot in annotations.items():
    all_annots.append(annot)

In [9]:
np.array(all_annots).sum(axis=1).min()

1.0

In [10]:
TEST = .5
CAL = .25
RES = .25


In [11]:
all_images = os.listdir(PATH)


n_test = int(len(all_images) * TEST)
n_cal = int(len(all_images) * CAL)


In [12]:
np.random.seed(42)
test_images = np.random.choice(all_images, n_test, replace=False)
all_images = list(set(all_images) - set(test_images))
cal_images = np.random.choice(all_images, n_cal, replace=False)
res_images = list(set(all_images) - set(cal_images))

In [13]:
len(test_images), len(cal_images), len(res_images)

(20252, 10126, 10126)

In [14]:
assert len(set(test_images) & set(cal_images)) == 0
assert len(set(test_images) & set(res_images)) == 0
assert len(set(cal_images) & set(res_images)) == 0

In [15]:
for img_name in tqdm(test_images):
    if img_name in annotations:
        if not os.path.exists(os.path.join(OUTPUT_PATH, "test", "images", img_name)):
            os.makedirs(os.path.join(OUTPUT_PATH, "test", "images"), exist_ok=True)
            os.makedirs(os.path.join(OUTPUT_PATH, "test", "labels"), exist_ok=True)
        shutil.copy(os.path.join(PATH, img_name), os.path.join(OUTPUT_PATH, "test", "images", img_name))
        np.save(os.path.join(OUTPUT_PATH, "test", "labels", img_name.replace(".jpg", ".npy")), annotations[img_name])

100%|██████████| 20252/20252 [00:06<00:00, 3294.69it/s]


In [16]:
for img_name in tqdm(cal_images):
    if img_name in annotations:
        if not os.path.exists(os.path.join(OUTPUT_PATH, "cal", "images", img_name)):
            os.makedirs(os.path.join(OUTPUT_PATH, "cal", "images"), exist_ok=True)
            os.makedirs(os.path.join(OUTPUT_PATH, "cal", "labels"), exist_ok=True)
        shutil.copy(os.path.join(PATH, img_name), os.path.join(OUTPUT_PATH, "cal", "images", img_name))
        np.save(os.path.join(OUTPUT_PATH, "cal", "labels", img_name.replace(".jpg", ".npy")), annotations[img_name])

100%|██████████| 10126/10126 [00:04<00:00, 2214.05it/s]


In [17]:
for img_name in tqdm(res_images):
    if img_name in annotations:
        if not os.path.exists(os.path.join(OUTPUT_PATH, "res", "images", img_name)):
            os.makedirs(os.path.join(OUTPUT_PATH, "res", "images"), exist_ok=True)
            os.makedirs(os.path.join(OUTPUT_PATH, "res", "labels"), exist_ok=True)
        shutil.copy(os.path.join(PATH, img_name), os.path.join(OUTPUT_PATH, "res", "images", img_name))
        np.save(os.path.join(OUTPUT_PATH, "res", "labels", img_name.replace(".jpg", ".npy")), annotations[img_name])

100%|██████████| 10126/10126 [00:02<00:00, 3676.69it/s]


In [19]:
mean_R = []
mean_G = []
mean_B = []
import cv2
for img_name in tqdm(os.listdir(os.path.join(OUTPUT_PATH, "res", "images"))):
    img = cv2.imread(os.path.join(OUTPUT_PATH, "res", "images", img_name))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    mean_R.append(img[:, :, 0].mean())
    mean_G.append(img[:, :, 1].mean())
    mean_B.append(img[:, :, 2].mean())

100%|██████████| 10028/10028 [00:56<00:00, 177.55it/s]


In [20]:
np.mean(mean_R), np.mean(mean_G), np.mean(mean_B)

(119.58259800370193, 114.03284258823422, 104.23619362025266)

In [23]:
np.save(os.path.join(OUTPUT_PATH, "mean.npy"), np.array([np.mean(mean_R), np.mean(mean_G), np.mean(mean_B)]))